# Geometric Transformations in 2D

SCC0250 - Computer Graphics
Fernando Gonçalves Campos - 12542352

**Controls**\
W/↑             acceleration\
S/↓             deceleration\
A/←             left rotation (only when mouse following is disabled)\
D/→             right rotation (only when mouse following is disabled)\
SPACE           brake\
+/-/scroll      change model size\
left mouse      moves the model to the mouse location and disables mouse following while held\
right mouse     disables mouse following while held

### Imports

In [20]:
import glfw
from OpenGL.GL import *
import glm

import numpy as np
import OpenGL.arrays.numpymodule as glnp

### Create Window

In [21]:
class WindowDim:
    '''
    Information about window dimensions
    '''
    width: int = 2
    height: int = 2
    left_limit: float = -1
    right_limit: float = 1
    bottom_limit: float = -1
    top_limit: float = 1

    @classmethod
    def update(cls, width, height) -> None:
        cls.width = width
        cls.height = height

        cls.left_limit = -width / 2
        cls.right_limit = width / 2

        cls.bottom_limit = -height / 2
        cls.top_limit = height / 2

In [22]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

WindowDim.update(1600, 800)

window = glfw.create_window(WindowDim.width, WindowDim.height, "Geometric Transformations - 12542352", None, None)
glfw.make_context_current(window)

### Shaders

In [23]:
# Allows transparency
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

In [24]:
vertex_code = """
    layout(location = 0) in vec4 attr_position;
    layout(location = 1) in vec4 attr_color;

    out vec4 vert_color;

    uniform mat4 MVP;

    void main() {
        gl_Position = MVP * attr_position;
        vert_color = attr_color;
    };
"""

fragment_code = """
    layout(location = 0) out vec4 out_color;

    in vec4 vert_color;

    void main() {
        out_color = vert_color;
    };
"""

In [25]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

In [26]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [27]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


In [28]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [29]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

In [30]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)

### Model Vertices

In [31]:
vertices = np.zeros(4, [("attr_position", np.float32, 3), ("attr_color", np.float32, 4)])
indices = np.zeros(6, [('indices', np.uint32)])

In [32]:
vertices['attr_position'] = [
    [120,   0, 0],
    [-45,   0, 0],
    [-80, -90, 0],
    [-80,  90, 0]
]

vertices['attr_color'] = [
    (1,    0,    0, 0.9),
    (1, 0.56,    0, 0.9),
    (1,    1, 0.28, 0.9),
    (1,    1, 0.28, 0.9)
]

indices['indices'] = [
    0, 1, 2,
    0, 1, 3
]

In [33]:
vertex_buffer = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, vertex_buffer)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)

index_buffer = glGenBuffers(1)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, index_buffer)
glBufferData(GL_ELEMENT_ARRAY_BUFFER, indices.nbytes, indices, GL_STATIC_DRAW)

In [34]:
stride = vertices.strides[0]
for i, (attr_name, (data_type, offset)) in enumerate(vertices.dtype.fields.items()):
    loc = glGetAttribLocation(program, attr_name)
    glEnableVertexAttribArray(loc)
    print(data_type.shape[0], offset)
    glVertexAttribPointer(
        loc,                                                    # location
        data_type.shape[0],                                     # size
        glnp.ARRAY_TO_GL_TYPE_MAPPING[data_type.subdtype[0]],   # type
        False,                                                  # normalized
        stride,                                                 # stride
        ctypes.c_void_p(offset)                                 # offset
    )

3 0
4 12


### Movement Control

In [35]:
class KeyInputs:
    SPACE: int = 0
    W: int = 0
    A: int = 0
    S: int = 0
    D: int = 0
    UP: int = 0
    LEFT: int = 0
    DOWN: int = 0
    RIGHT: int = 0
    MINUS: int = 0
    PLUS: int = 0
    MINUS_KP: int = 0
    PLUS_KP: int = 0

    @classmethod
    def get_inputs(cls, window) -> None:
        cls.SPACE = glfw.get_key(window, glfw.KEY_SPACE)
        cls.W = glfw.get_key(window, glfw.KEY_W)
        cls.A = glfw.get_key(window, glfw.KEY_A)
        cls.S = glfw.get_key(window, glfw.KEY_S)
        cls.D = glfw.get_key(window, glfw.KEY_D)
        cls.UP = glfw.get_key(window, glfw.KEY_UP)
        cls.LEFT = glfw.get_key(window, glfw.KEY_LEFT)
        cls.DOWN = glfw.get_key(window, glfw.KEY_DOWN)
        cls.RIGHT = glfw.get_key(window, glfw.KEY_RIGHT)
        cls.MINUS = glfw.get_key(window, glfw.KEY_MINUS)
        cls.KP_MINUS = glfw.get_key(window, glfw.KEY_KP_SUBTRACT)
        cls.PLUS = glfw.get_key(window, glfw.KEY_EQUAL)
        cls.KP_PLUS = glfw.get_key(window, glfw.KEY_KP_ADD)

class MouseInputs:
    X_POS: float = 0
    Y_POS: float = 0
    LBUTTON: int = 0
    RBUTTON: int = 0
    MBUTTON: int = 0
    SCROLL_X: float = 0.0
    SCROLL_Y: float = 0.0

    @classmethod
    def get_inputs(cls, window) -> None:
        # Converts coordinate system used in glfw to the coordinate system used to work with the model
        x_pos, y_pos = glfw.get_cursor_pos(window)
        cls.X_POS = x_pos + WindowDim.left_limit
        cls.Y_POS = WindowDim.top_limit - y_pos

        cls.LBUTTON = glfw.get_mouse_button(window, glfw.MOUSE_BUTTON_LEFT)
        cls.RBUTTON = glfw.get_mouse_button(window, glfw.MOUSE_BUTTON_RIGHT)
        cls.MBUTTON = glfw.get_mouse_button(window, glfw.MOUSE_BUTTON_MIDDLE)

    @classmethod
    def get_scroll(cls, window, scroll_x, scroll_y) -> None:
        cls.SCROLL_Y = scroll_y
        cls.SCROLL_X = scroll_x

glfw.set_scroll_callback(window, MouseInputs.get_scroll)

In [36]:
class ModelMovements:
    def __init__(
        self,
        starting_x: float = 0.0,
        starting_y: float = 0.0,
        starting_angle: float = 0.0,
        starting_scale_x: float = 1.0,
        starting_scale_y: float = 1.0,
        acceleration: float = 400,
        deceleration: float = 400,
        ang_speed: float = 2,
        scale_speed: float = 6,
    ):
        self.x = starting_x
        self.y = starting_y
        self.angle = starting_angle
        self.scale_x = starting_scale_x
        self.scale_y = starting_scale_y

        self.speed_x = 0.0
        self.speed_y = 0.0
        self.ang_speed = ang_speed

        self.scale_speed = scale_speed

        self.acceleration = acceleration
        self.deceleration = deceleration

    def handle_events(self, delta_time: float) -> None:
        self._handle_mouse_events(delta_time)
        self._handle_keyboard_events(delta_time)
        self._handle_warping()

        # Handle brake
        if KeyInputs.SPACE == glfw.PRESS:
            self.speed_x = 0.0
            self.speed_y = 0.0

    def _handle_keyboard_events(self, delta_time: float) -> None:
        # Scale
        if KeyInputs.MINUS == glfw.PRESS or KeyInputs.KP_MINUS == glfw.PRESS:
            self.scale_x -= self.scale_speed * delta_time
            self.scale_y -= self.scale_speed * delta_time
            self.scale_x = max(self.scale_x, 0)
            self.scale_y = max(self.scale_y, 0)

        if KeyInputs.PLUS == glfw.PRESS or KeyInputs.KP_PLUS == glfw.PRESS:
            self.scale_x += self.scale_speed * delta_time
            self.scale_y += self.scale_speed * delta_time

        # Rotation
        # When the effect of following the mouse is disabled, enables manual rotation
        if MouseInputs.LBUTTON == glfw.PRESS or MouseInputs.RBUTTON == glfw.PRESS:
            if KeyInputs.LEFT == glfw.PRESS or KeyInputs.A == glfw.PRESS:
                self.angle += self.ang_speed * delta_time
            if KeyInputs.RIGHT == glfw.PRESS or KeyInputs.D == glfw.PRESS:
                self.angle -= self.ang_speed * delta_time

        # Propulsion
        x_factor = glm.cos(self.angle)
        y_factor = -glm.sin(self.angle)

        half_x_speed_inc = 0
        half_y_speed_inc = 0
        if KeyInputs.UP == glfw.PRESS or KeyInputs.W == glfw.PRESS:
            half_x_speed_inc += self.acceleration * delta_time * x_factor / 2
            half_y_speed_inc += self.acceleration * delta_time * y_factor / 2

        if KeyInputs.DOWN == glfw.PRESS or KeyInputs.S == glfw.PRESS:
            half_x_speed_inc -= self.deceleration * delta_time * x_factor / 2
            half_y_speed_inc -= self.deceleration * delta_time * y_factor / 2

        # Increasing speed in this way for consistency in different frame rates
        self.speed_x += half_x_speed_inc
        self.speed_y -= half_y_speed_inc

        self.x += self.speed_x * delta_time
        self.y += self.speed_y * delta_time

        self.speed_x += half_x_speed_inc
        self.speed_y -= half_y_speed_inc

    def _handle_mouse_events(self, delta_time: float) -> None:
        # Scale
        self.scale_x += self.scale_speed * MouseInputs.SCROLL_Y / 20
        self.scale_y += self.scale_speed * MouseInputs.SCROLL_Y / 20
        self.scale_x = max(self.scale_x, 0)
        self.scale_y = max(self.scale_y, 0)

        # Grab
        if MouseInputs.LBUTTON == glfw.PRESS:
            self.x = MouseInputs.X_POS
            self.y = MouseInputs.Y_POS

        # Rotation
        elif MouseInputs.RBUTTON != glfw.PRESS:
            relative_x: glm.float32 = MouseInputs.X_POS - self.x
            relative_y: glm.float32 = MouseInputs.Y_POS - self.y

            if relative_x == 0:
                if relative_y != 0:
                    self.angle = glm.sign(relative_y) * glm.pi() / 2
            else:
                left_direction = 0 if relative_x >= 0 else glm.pi() # atan only calculates angles with positive cos
                self.angle = glm.atan(relative_y / relative_x) + left_direction

    def _handle_warping(self) -> None:
        # While loops needed for speeds larger than the window size
        if self.x > WindowDim.right_limit:
            while self.x - WindowDim.width >= WindowDim.left_limit:
                self.x -= WindowDim.width
        if self.x < WindowDim.left_limit:
            while self.x + WindowDim.width <= WindowDim.right_limit:
                self.x += WindowDim.width

        if self.y > WindowDim.top_limit:
            while self.y - WindowDim.height >= WindowDim.bottom_limit:
                self.y -= WindowDim.height
        if self.y < WindowDim.bottom_limit:
            while self.y + WindowDim.height <= WindowDim.top_limit:
                self.y += WindowDim.height

    def get_model_matrix(self) -> glm.mat4:
        model_mat = glm.mat4(1)
        model_mat = glm.translate(model_mat, glm.vec3(self.x, self.y, 0))
        model_mat = glm.rotate(model_mat, self.angle, glm.vec3(0, 0, 1))
        model_mat = glm.scale(model_mat, glm.vec3(self.scale_x, self.scale_y, 1))
        return model_mat

movements = ModelMovements()

### Main Loop

In [37]:
bg_color: glm.vec3 = glm.vec3(1,1,1)

view_mat: glm.mat4 = glm.mat4(1.0)

# Converts values used for calculating rendering locations (in pixels) to the normalized range [-1, 1] used by OpenGL
proj_mat: glm.mat4 = glm.ortho(WindowDim.left_limit, WindowDim.right_limit, WindowDim.bottom_limit, WindowDim.top_limit, -1, 1)

def handle_window_resizing(window, width, height):
    WindowDim.update(width, height)

    global proj_mat
    proj_mat = glm.ortho(WindowDim.left_limit, WindowDim.right_limit, WindowDim.bottom_limit, WindowDim.top_limit, -1, 1)

    glViewport(0, 0, width, height)

glfw.set_framebuffer_size_callback(window, handle_window_resizing)

glfw.show_window(window)

# Used for keeping movement independent from the frame rate
current_time: float = glfw.get_time()
previous_time: float = current_time
delta_time: float = 0

while not glfw.window_should_close(window):
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(bg_color.x, bg_color.y, bg_color.z, 1.0)

    glfw.get_window_size(window)
    current_time = glfw.get_time()
    delta_time = current_time - previous_time
    previous_time = current_time

    # Resets the scroll distance to avoid infinite scaling
    MouseInputs.SCROLL_Y = 0
    MouseInputs.SCROLL_X = 0
    glfw.poll_events()
    KeyInputs.get_inputs(window)
    MouseInputs.get_inputs(window)
    movements.handle_events(delta_time)

    model_mat: glm.mat4 = movements.get_model_matrix()

    # Renders the model in its surroundings to enhance the warping effect
    for i in range(-1, 2):
        for j in range(-1, 2):
            cur_model_mat: glm.mat4 = glm.translate(glm.mat4(1.0), glm.vec3(j * WindowDim.width, i * WindowDim.height, 0))
            cur_model_mat = cur_model_mat * model_mat
            MVP_mat: glm.mat4 = proj_mat * view_mat * cur_model_mat

            loc = glGetUniformLocation(program, 'MVP')
            glUniformMatrix4fv(loc, 1, GL_FALSE, glm.value_ptr(MVP_mat))
            glDrawElements(GL_TRIANGLES, 6, GL_UNSIGNED_INT, None)

    glfw.swap_buffers(window)

glfw.terminate()